# 1. Introduction 

<h3> Scenario: <h3>   
<h4> I currently live in Los Angeles, CA, but have been offered a dream job in New York. Before I move overseas, I want to explore where I can live that can resemble my life in LA. The general question that needs to be answered is: Can I find a place that fits my criteria in New York City?<h4>

#### Criteria:
     - 2 Bedrooms
     - Within 2 miles from a metro station
     -Venues within 10 miles, Coffee Shop, Gym, Grocery Store, and Italian Resturaunt
     - Rent can't exceed $3,000 a month
  

# 2. Data Section

## Required data:
    -List of neighborhoods in Paris with geodata
    -List of metro stations in Paris
    -List of apartments with address and price
    -List of venues

In [9]:
import numpy as np
import time
import pandas as pd
import json
import requests
from pandas.io.json import json_normalize
from geopy.geocoders import Nominatim
import folium

In [5]:
neighborhood_latitude = 33.993126
neighborhood_longitude = -118.4562
print(f'The coordinates of the address are {neighborhood_latitude}, {neighborhood_longitude}')

The coordinates of the address are 33.993126, -118.4562


In [4]:
CLIENT_ID = 'DJ0WRSIY3VWEN2KS2GLS4NM2GW0WYU2RZZ1W0U4BLIXGT02E'
CLIENT_SECRET = 'KETE3K5PY1RNSXB5FOCYHUFQF0WGOHTUQQRIZ1Z13CBNYUTG'
VERSION = '20200701'

In [6]:
LIMIT = 100
radius = 400

url ='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=DJ0WRSIY3VWEN2KS2GLS4NM2GW0WYU2RZZ1W0U4BLIXGT02E&client_secret=KETE3K5PY1RNSXB5FOCYHUFQF0WGOHTUQQRIZ1Z13CBNYUTG&v=20200701&ll=33.993126,-118.4562&radius=400&limit=100'

In [7]:
results = requests.get(url).json()

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [10]:
venues = results['response']['groups'][0]['items']

SGnearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
SGnearby_venues =SGnearby_venues.loc[:, filtered_columns]

# filter the category for each row
SGnearby_venues['venue.categories'] = SGnearby_venues.apply(get_category_type, axis=1)

# clean columns
SGnearby_venues.columns = [col.split(".")[-1] for col in SGnearby_venues.columns]

SGnearby_venues.head(10)

,name,categories,lat,lng
0,Superba Food + Bread,New American Restaurant,33.996237,-118.456471
1,Deus Ex Machina,Café,33.994371,-118.452933
2,Love Shack,Furniture / Home Store,33.995013,-118.455144
3,First Friday on Abbot Kinney,Food Truck,33.994324,-118.453510
4,Bruno's Dog and Cat Boutique,Pet Store,33.995598,-118.455458
5,General Store,Boutique,33.996355,-118.457662
6,Deus Espresso Bar,Coffee Shop,33.994384,-118.453005
7,Late Sunday Afternoon,Gift Shop,33.995930,-118.456170
8,Tradesmen,Men's Store,33.996125,-118.457561
9,Wirin Thai,Thai Restaurant,33.994455,-118.453144


# Create map of Venice, CA with the above geodata for the venues

In [14]:
map_la = folium.Map(location = [latitude, longitude], zoom_start=20)

for lat, lng, label in zip(LAnearby_venues['lat'], LAnearby_venues['lng'], LAnearby_venues['name']):
    label = folium.Popup(label, parse_html = True)
    folium.RegularPolygonMarker(
        [lat,lng],
        number_of_sides=4,
        radius=10,
        popup=label,
        color='blue').add_to(map_la)
    
map_la

# Place Manhattan neighborhoods in dataframe